In [1]:
import sys
import os
sys.path.append("..")
from globals import ROOT_DIR
from data_providers import TextDataProvider
import configparser
import argparse
import configparser
from torch import optim
from experiment_builder import ExperimentBuilder
from data_providers import *
import os
from models.cnn import *
from models.multilayer_perceptron import multi_layer_perceptron
config = configparser.ConfigParser()
config.read('../config.ini')
path_data = os.path.join(ROOT_DIR, config['DEFAULT']['PATH_DATA'])
path_labels = os.path.join(ROOT_DIR, config['DEFAULT']['PATH_LABELS'])

In [2]:
VERBOSE = True
def extract_data(embedding_key, embedding_level_key, seed):
    path_data = os.path.join(ROOT_DIR, config['DEFAULT']['PATH_DATA'])
    path_labels = os.path.join(ROOT_DIR, config['DEFAULT']['PATH_LABELS'])
    data_provider = TextDataProvider(path_data, path_labels)
    if embedding_level_key == 'word':
        output = data_provider.generate_word_level_embeddings(embedding_key, seed)
    elif embedding_level_key == 'char':
        output = data_provider.generate_char_level_embeddings(seed)
    else:
        output = data_provider.generate_tdidf_embeddings(seed)

    if VERBOSE:
        print("[Sizes] Training set: {}, Validation set: {}, Test set: {}".format(len(output['x_train']),
                                                                                  len(output['x_valid']),
                                                                                  len(output['x_test'])))
    return output

In [3]:
data = extract_data(embedding_key='twitter', embedding_level_key='word', seed=28)

=== Extracting annotations ===
=== Extracting tweets from JSON ===
[Stats] Removed 3/58358 labels
[Stats] Average tweet length is 17 words
[Stats] Average tweet length is 121 characters
[Stats] Average favorite count is 15
[Stats] Average retweet count is 146
[Stats] Average follower count is 710
[Sizes] Training set: 64.00%, Validation set: 16.00%, Test set: 20.00%
[Model] Using twitter embeddings
[Sizes] Training set: 64.00%, Validation set: 16.00%, Test set: 20.00%
[Sizes] Training set: 37348, Validation set: 9338, Test set: 11672


In [4]:
data['x_train'][0]

[array([ 1.36505123e+00, -3.69938859e-01, -1.40007032e-01, -5.52433503e-01,
        -2.39834834e-01,  3.53300874e-02, -2.42736525e-01,  1.21473319e-01,
        -4.50574352e-01, -1.54782067e-02,  1.84717461e-01, -8.46866535e-01,
        -3.21081362e-01, -1.13971723e-01, -2.12251571e-01, -1.34926561e+00,
         2.11117509e-01,  6.42623766e-02, -4.99484138e-01,  1.65180133e+00,
        -3.56855288e-01,  6.76405702e-01,  2.44145639e-01, -5.95142679e-02,
         3.11052659e-01, -1.30082188e+00,  1.10972449e-01, -4.98668353e-01,
        -6.06137791e-02, -1.12081288e+00,  7.38298546e-01, -2.47751260e-02,
         5.16686256e-01,  7.55475698e-01,  1.27145999e+00,  6.80079369e-02,
         2.77755786e-01, -3.49561130e-01, -3.73726124e-01, -2.02361846e-01,
        -6.13653561e-02,  5.68230174e-01, -1.08273482e-01,  5.96918680e-01,
         4.14588522e-01,  1.38932747e-01,  4.31445363e-01, -1.38229707e+00,
         6.22547052e-01,  2.44586929e-02,  4.28718467e-01, -1.35659762e-01,
         9.9

In [ ]:
def wrap_data(batch_size, seed, x_train, y_train, x_valid, y_valid, x_test, y_test):
    train_set = DataProvider(inputs=x_train, targets=y_train, seed=seed)
    train_data_local = torch.utils.data.DataLoader(train_set,
                                                   batch_size=batch_size,
                                                   num_workers=2,
                                                   sampler=ImbalancedDatasetSampler(train_set))

    valid_set = DataProvider(inputs=x_valid, targets=y_valid, seed=seed)
    valid_data_local = torch.utils.data.DataLoader(valid_set,
                                                   batch_size=batch_size,
                                                   num_workers=2,
                                                   shuffle=False)

    test_set = DataProvider(inputs=x_test, targets=y_test, seed=seed)
    test_data_local = torch.utils.data.DataLoader(test_set,
                                                  batch_size=batch_size,
                                                  num_workers=2,
                                                  shuffle=False)

    return train_data_local, valid_data_local, test_data_local

def fetch_model(model, embedding_level, input_shape_local, dropout):
    if model == 'MLP':
        return multi_layer_perceptron(input_shape_local)
    if model == 'CNN':
        if embedding_level == 'word':
            return word_cnn(input_shape_local, dropout)
        elif embedding_level == 'character':
            return character_cnn(input_shape_local)
    if model == 'DENSENET':
        return densenet()
    else:
        raise ValueError("Model key not found {}".format(embedding_level))


def fetch_model_parameters(input_shape_local):
    model_local, criterion_local, optimizer_local = fetch_model(model='CNN',
                                                                embedding_level='word',
                                                                input_shape_local=input_shape_local,
                                                                dropout=0.5)

    scheduler_local = optim.lr_scheduler.CosineAnnealingLR(optimizer_local, T_max=100, eta_min=0.0001)
    return model_local, criterion_local, optimizer_local, scheduler_local



In [ ]:
train_data, valid_data, test_data = wrap_data(64, 28, **data)

In [6]:
input_shape = tuple([64] + list(np.array(data['x_train']).shape)[1:])
input_shape

(64, 17, 400)

In [ ]:
def run_train_iter(model, device, optimizer, criterion, x, y, stats, experiment_key='train'):
    """
    Receives the inputs and targets for the model and runs a training iteration. Returns loss and accuracy metrics.
    :param x: The inputs to the model. A numpy array of shape batch_size, channels, height, width
    :param y: The targets for the model. A numpy array of shape batch_size, num_classes
    :return: the loss and accuracy for this batch
    """
    # sets model to training mode
    # (in case batch normalization or other methods have different procedures for training and evaluation)
    model.train()
    x = x.float()
    x = x.to(device)
    y = y.to(device)
    optimizer.zero_grad()  # set all weight grads from previous training iters to 0
    out = model.forward(x)  # forward the data in the model
    # loss = F.cross_entropy(input=out, target=y)  # compute loss
    loss = criterion(out, y)
    loss.backward()  # backpropagate to compute gradients for current iter loss

    optimizer.step()  # update network parameters
    _, predicted = torch.max(out.data, 1)  # get argmax of predictions
    accuracy = np.mean(list(predicted.eq(y.data).cpu()))  # compute accuracy
    stats['{}_acc'.format(experiment_key)].append(accuracy)
    stats['{}_loss'.format(experiment_key)].append(loss.data.detach().cpu().numpy())

def run_evaluation_iter(model, device, optimizer, criterion, x, y, stats, experiment_key='valid'):
    """
    Receives the inputs and targets for the model and runs an evaluation iterations. Returns loss and accuracy metrics.
    :param x: The inputs to the model. A numpy array of shape batch_size, channels, height, width
    :param y: The targets for the model. A numpy array of shape batch_size, num_classes
    :return: the loss and accuracy for this batch
    """
    with torch.no_grad():
        model.eval()  # sets the system to validation mode
        x = x.float()
        x = x.to(device)
        y = y.to(device)
        out = model.forward(x)  # forward the data in the model
        loss = criterion(out, y)
        
        # loss = F.cross_entropy(out, y)  # compute loss
        _, predicted = torch.max(out.data, 1)  # get argmax of predictions
        
        accuracy = np.mean(list(predicted.eq(y.data).cpu()))
        stats['{}_acc'.format(experiment_key)].append(accuracy)  # compute accuracy
        stats['{}_loss'.format(experiment_key)].append(loss.data.detach().cpu().numpy())
        

In [ ]:
def save_model(model, model_save_dir, model_save_name, model_idx):
    """
    Save the network parameter state and current best val epoch idx and best val accuracy.
    :param model_save_name: Name to use to save model without the epoch index
    :param model_idx: The index to save the model with.
    :param best_validation_model_idx: The index of the best validation model to be stored for future use.
    :param best_validation_model_acc: The best validation accuracy to be stored for use at test time.
    :param model_save_dir: The directory to store the state at.
    :param state: The dictionary containing the system state.

    """
    # Save state each epoch
    path = os.path.join(model_save_dir, "{}_{}".format(model_save_name, str(model_idx)))
    torch.save(model.state_dict(), f=path)
    

def load_model(model, model_save_dir, model_save_name, model_idx):
    """
    Load the network parameter state and the best val model idx and best val acc to be compared with the future val accuracies, in order to choose the best val model
    :param model_save_dir: The directory to store the state at.
    :param model_save_name: Name to use to save model without the epoch index
    :param model_idx: The index to save the model with.
    """
    path = os.path.join(model_save_dir, "{}_{}".format(model_save_name, str(model_idx)))
    checkpoint = torch.load(f=path)
    # freeze parameters
    model.load_state_dict(checkpoint)
    for parameter in model.parameters():
        parameter.requires_grad = False
    return model 

In [ ]:
from collections import OrderedDict, defaultdict
import tqdm

model, criterion, optimizer, _ = fetch_model_parameters(input_shape)
device = torch.device('cpu')
train_stats = OrderedDict()
model_params = {}
num_epochs = 2
for epoch_idx in range(num_epochs):
    epoch_start_time = time.time()
    epoch_stats = defaultdict(list)
    with tqdm.tqdm(total=len(train_data)) as pbar_train:  # create a progress bar for training
        for idx, (x, y) in enumerate(train_data):  # get data batches
            run_train_iter(model, device, optimizer, criterion, x=x, y=y, stats=epoch_stats)  # take a training iter step
            pbar_train.update(1)
            pbar_train.set_description("loss: {:.4f}, accuracy: {:.4f}".format(epoch_stats['train_loss'][-1],
                                                                               epoch_stats['train_acc'][-1]))

    with tqdm.tqdm(total=len(valid_data)) as pbar_val:  # create a progress bar for validation
        for x, y in valid_data:  # get data batches
            run_evaluation_iter(model, device, optimizer, criterion, x=x, y=y, stats=epoch_stats)  # run a validation iter
            pbar_val.update(1)  # add 1 step to the progress bar
            pbar_val.set_description("loss: {:.4f}, accuracy: {:.4f}".format(epoch_stats['valid_loss'][-1],
                                                                             epoch_stats['valid_acc'][-1]))
    #evaluate test here
    with tqdm.tqdm(total=len(test_data)) as pbar_test:  # create a progress bar for validation
        for x, y in test_data:  # get data batches
            run_evaluation_iter(model, device, optimizer, criterion, x=x, y=y, stats=epoch_stats, experiment_key="test_experiment")  # run a validation iter
            pbar_test.update(1)  # add 1 step to the progress bar
            pbar_test.set_description("loss: {:.4f}, accuracy: {:.4f}".format(epoch_stats['test_experiment_loss'][-1],
                                                                  epoch_stats['test_experiment_acc'][-1]))
    
    model_params[epoch_idx] = list(model.parameters())
    save_model(model, '', 'testing', epoch_idx)

In [ ]:
loaded_model = load_model(model, '', 'testing', 1)

In [ ]:
loaded_model.state_dict()

In [ ]:
model.state_dict()

In [ ]:
np.array(data['x_train'][0]).float()

In [ ]:
torch.equal(list(loaded_model.parameters())[0], model_params[0][0])

In [ ]:
torch.equal(model_first_parameters[0], final_model_parameters[0])

In [ ]:
for parameter in model.parameters():
    parameter.requires_grad = False

In [ ]:
torch.equal(model_first_parameters[0], final_model_parameters[0])

In [ ]:
final_model_parameters = list(model.parameters())


In [ ]:
torch.equal(model_first_parameters[0], final_model_parameters[0])

In [ ]:
model.layer_dict

In [ ]:
model.state_dict()